# Homework 1

## 1.1

    I don't think that introspection would be a very good guide for modeling human cognitive processes. Introspection is internally-based, meaning that it gets its information from inside itself, which doesn't really work for an artificial intelligence, given that it requires information from outside the brain to learn from in the first place.
    Introspection also inherently involves self-consciousness, which isn't something an artificial intelligence is capable of. Introspection also focuses on thoughts and feelings, which requires the capacity for emotions, which artificial intelligence lacks. Altogether, artificial intelligence lacks the self-awareness that is the basis of introspection, and that is very possibly a good thing, since a truly self-aware artificial intelligence would be rather concerning.

## 1.2

TSP Problem Formulation:
State representation: 
        [c1, c2, ..., cn] gives the order in which the cities are visited.
    Move representation: 
        [num1, num2]: Swap the positions (within the list) of the cities at the given indexes.

"""
This implements a local-search version of a TSP problem for AIMA-Python.
Based on kvlinden's queens.py.

@authors: ljh27 & slg27
"""
from tools.aima.search import Problem, hill_climbing, simulated_annealing, exp_schedule
from random import randrange


class TSP(Problem):
    """An implementation of a TSP problem for local search.

    State representation: 
        [c1, c2, ..., cn] gives the order in which the cities are visited.
    Move representation: 
        [num1, num2]: Swap the positions (within the list) of the cities at the given indexes.
    """

    def __init__(self, distances, initial):
        self.distances = distances
        self.initial = initial

    def actions(self, state):
        """This method generates two random integers to be used as indexes in a move.
        """
        num_cities = len(state) - 1 #minus 1 because start and end in same city
        actions = []
        for i in range(6):
            num1 = randrange(1, num_cities)
            num2 = randrange(1, num_cities)
            action = [num1, num2]
            actions.append(action)
        return actions

    def result(self, state, move):
        """Makes the given move (swaps the positions of the cities at the given
        indexes) on a copy of the given state."""
        new_state = state[:]
        new_state[move[0]], new_state[move[1]] = new_state[move[1]], new_state[move[0]]
        return new_state

    def value(self, state):
        """This method computes the total distance traveled in the given state by adding
        up the distance of each segment, then multiplying it by -1 to more semantically
        reflect that greater distance is less ideal.
        """
        num_visits = len(state)
        value = 0

        for i in range(1, num_visits):
            try:
                value += distances[(state[i-1], state[i])]
            except KeyError:
                value += distances[(state[i], state[i - 1])]

        value = -1 * value

        return value

My sample city-domain is made up of 5 cities represented by the letters A-E.
The two algorithms do sometimes reach the same result, but overall the simulated annealing algorithm tends to find a better solution than the hill-climbing algorithm does because it "looks ahead" more and is thus willing to move to a less-ideal state in order to progress into a better state from there, whereas the hill-climbing algorithm only ever takes steps that are immediately beneficial.

In [ ]:
if __name__ == '__main__':

    # Initialize a path and a dictionary of distances between cities
    initial_path = ['A', 'B', 'C', 'D', 'E', 'A']
    distances = {('A', 'B'): 1, ('A', 'C'): 3, ('A', 'D'): 2, ('A', 'E'): 4, ('B', 'C'): 3, ('B', 'D'): 2, ('B', 'E'): 1, ('C', 'D'): 8, ('C', 'E'): 3, ('D', 'E'): 5}

    # Initialize the TSP problem
    p = TSP(distances, initial_path)

    # Solve the problem using hill climbing.
    hill_solution = hill_climbing(p)
    print('Hill-climbing:')
    print('\tSolution: ' + str(hill_solution))
    print('\tDistance:    ' + str(p.value(hill_solution)))

    # Solve the problem using simulated annealing.
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=10000)
    )
    print('Simulated annealing:')
    print('\tSolution: ' + str(annealing_solution))
    print('\tDistance:    ' + str(p.value(annealing_solution)))

## 1.3

The following code implements a course-scheduling CSP. As we discussed in class, the courses to be scheduled are the variables, the domains are represented in a dictionary in which the keys are the courses and the values are lists of tuples containing the possible combinations of professor, classroom, and class time. Every class is a neighbor of every other class. The constraint function checks if there are any conflicts between professors and times or between times and rooms.

In order to implement this scheduler CSP, I wrote 3 new methods: the first generates all the possible combinations of the given classrooms and class times, which the second method uses to generate the domains dictionary. The third generates a dictionary specifying the neighbors of each course, which are used to evaluate the constraints using the schedule_constraint method. The third new method I added generates the neighbors non-reflexively, since it would be redundant to compare, for instance, cs108 to cs112 if you had already compared cs112 to cs108. I chose this way of formulating/implementing the problem because it fit with what we talked about in class, and it was the way that things made the most clear sense to me.

In [2]:
"""scheduler.py implements a course-scheduling CSP.
    variables   A list of the courses to be scheduled
    domains     A dictionary in which the keys are the courses and the values are triples containing the possible
                professor/room/time combinations
    neighbors   All course pairs
    constraint_function: a function that returns false if the given variable values are conflicting
"""

from tools.aima.csp import CSP, min_conflicts


def create_timeroom_slots(times, rooms):
    timeroom_slots = []
    for time in times:
        for room in rooms:
            timeroom_slots.append((time, room))
    return timeroom_slots


def create_domains(courses, assignments, timeroom_slots):
    domains = {}
    for course in courses:
        dom = []
        prof = assignments[course]
        for slot in timeroom_slots:
            dom.append((prof, slot[0], slot[1]))
        domains[course] = dom
    return domains


def create_nonreflexive_neighbors(courses):
    neighbors = {}
    num_courses = len(courses)
    for i in range(num_courses):
        nbrs = []
        for j in range(i+1, num_courses):
            nbrs.append(courses[j])
        neighbors[courses[i]] = nbrs
    return neighbors


def schedule_constraint(course1, slot1, course2, slot2):
    #grab the details from each slot
    prof1 = slot1[0]
    time1 = slot1[1]
    room1 = slot1[2]
    prof2 = slot2[0]
    time2 = slot2[1]
    room2 = slot2[2]

    #make sure there aren't any time/room overlaps
    no_room_conflict = (time1 != time2) or (room1 != room2)

    #make sure there aren't any prof/time overlaps
    no_prof_conflict = (prof1 != prof2) or (time1 != time2)

    #only return true if there are no conflicts of either type
    return (no_room_conflict and no_prof_conflict)


#Set up some test data
courses = ['cs108', 'cs112', 'cs212', 'cs214', 'cs232', 'cs262', 'cs342']
faculty = ['adams', 'plantinga', 'bailey', 'schuurman']
assignments = {'cs108': 'schuurman', 'cs112': 'adams', 'cs212': 'plantinga', 'cs214': 'schuurman', 'cs232': 'adams', 'cs262': 'bailey', 'cs342': 'bailey'}
classtimes = ['mwf0900', 'mwf1030', 'mwf1130', 'mwf1230']
classrooms = ['nh253', 'sb382']
slots = create_timeroom_slots(classtimes, classrooms)
domains = create_domains(courses, assignments, slots)
neighbors = create_nonreflexive_neighbors(courses)

#Run it!
solution = min_conflicts(CSP(courses, domains, neighbors, schedule_constraint))

print('Variables: ' + str(courses))
print('Domains:')
for d in domains:
    print("\t{0}: {1}".format(d, domains[d]))
print('Neighbors (non-reflexive):')
for n in neighbors:
    print("\t{0}: {1}".format(n, neighbors[n]))
print('Solution:')
for s in solution:
    print("\t{0}: {1}".format(s, solution[s]))

ModuleNotFoundError: No module named 'tools'